# DANN

In [23]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function

from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms, datasets
from torchvision import models
from torchvision.models.feature_extraction import create_feature_extractor

In [24]:
train_transform = [transforms.ToTensor(), transforms.Normalize(0.5, 0.5), transforms.Resize((32,32))]
svhn_transform = [transforms.ToTensor(), transforms.Normalize(0.5, 0.5), transforms.Resize((32,32)), transforms.Grayscale(num_output_channels=1)]
train_mnist = datasets.MNIST(root='./data', train=True, transform=transforms.Compose(train_transform), download=True)
train_svhn = datasets.SVHN(root='./data', split='train', transform=transforms.Compose(svhn_transform), download=True)

mnist_loader = DataLoader(train_mnist, batch_size=128, shuffle=True)
svhn_loader = DataLoader(train_svhn, batch_size=128, shuffle=True)


Using downloaded and verified file: ./data/train_32x32.mat


## DANN Model

In [40]:
# class FeatureExtractor(nn.Module):
#     def __init__(self, in_dim, dim):
#         super().__init__()
#         self.block = nn.Sequential(
#             nn.Conv2d(in_channels=in_dim, out_channels=dim, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm2d(dim),
#             nn.LeakyReLU(0.2),
#             nn.Conv2d(in_channels=dim, out_channels=dim*2, kernel_size=3, stride=2, padding=1),
#             nn.BatchNorm2d(dim*2),
#             nn.LeakyReLU(0.2),
#             nn.Conv2d(in_channels=dim*2, out_channels=dim*4, kernel_size=3, stride=2, padding=1),
#             nn.BatchNorm2d(dim*4),
#             nn.LeakyReLU(0.2),
#             nn.Dropout2d()
#         )
#         self.avgpool = nn.AdaptiveAvgPool2d(output_size=1)

#     def forward(self, x):
#         bs = x.size(0)
#         x = self.block(x)
#         x = self.avgpool(x)
#         return x.view(bs, -1)

class DomainPredictor(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(in_dim, in_dim//2),
            nn.BatchNorm1d(in_dim//2),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(),
            nn.Linear(in_dim//2, 2),
            nn.LogSoftmax()
        )

    def forward(self, x):
        return self.block(x)

class LabelPredictor(nn.Module):
    def __init__(self, in_dim, class_num):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(in_dim, in_dim//2),
            nn.BatchNorm1d(in_dim//2),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(),
            nn.Linear(in_dim//2, class_num),
            nn.LogSoftmax()
        )

    def forward(self, x):
        return self.block(x)

class GradientReverse(Function):
    def __init__():
        super().__init__()

    @staticmethod
    def forward(ctx, input, alpha):
        # ctx.save_for_backward(input) # input을 빼줄 필요가 없음. 단순히 Gradient에 -alpha 곱해줄거라
        ctx.alpha = alpha
        return input

    def backward(ctx, grad_output):
        # input, = ctx.saved_tensors # input이 개입하지 않기 때문에 주석.

        return -grad_output * ctx.alpha , None # 코드에서는 None 를 추가로 빼준다. Grad output이 원래 2개의 값을 내줘야한다고 함.

class DANN(nn.Module):
    def __init__(self, class_num):
        super().__init__()
        # self.feature_extractor = FeatureExtractor(in_dim=1, dim=16)
        model = models.resnet50(pretrained=True)
        model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        return_nodes = {
            'avgpool': 'avgpool'
        }
        self.feature_extractor = create_feature_extractor(model, return_nodes=return_nodes)

        self.domain_predictor = DomainPredictor(in_dim=2048)
        self.label_predictor = LabelPredictor(in_dim=2048, class_num=class_num)
    def forward(self, x, alpha):
        bs = x.size(0)
        x = self.feature_extractor(x)['avgpool'].view(bs, -1)
        
        # https://pytorch.org/tutorials/beginner/examples_autograd/polynomial_custom_function.html
        reverse_x = GradientReverse.apply(x, alpha) # customized Gradient Apply
        label_pred = self.label_predictor(x)
        domain_pred = self.domain_predictor(reverse_x)
        
        return label_pred, domain_pred



## DANN Train

In [41]:

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = DANN(class_num=10).to(DEVICE)

optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)

l_crit = nn.NLLLoss()
d_crit = nn.NLLLoss()

for epoch in range(10):

    len_dataloader = min(len(mnist_loader), len(svhn_loader))
    data_target_iter = iter(mnist_loader)
    data_source_iter = iter(svhn_loader)

    i = 0
    
    while i < len_dataloader:
        optimizer.zero_grad()
        p = float(i + epoch * len_dataloader) / 50 / len_dataloader # moving
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        img, label = data_source_iter.next()
        domain_label = torch.zeros(label.size())
        if torch.cuda.is_available():
            img = img.to(DEVICE)
            label = label.to(DEVICE)
            domain_label = domain_label.long().to(DEVICE)
        
        l_pred, d_pred = model(img, alpha)

        src_l_loss = l_crit(l_pred, label)
        src_d_loss = d_crit(d_pred, domain_label)

        t_img, t_label = data_target_iter.next()
        domain_label = torch.ones(t_label.size())

        if torch.cuda.is_available():
            t_img = t_img.to(DEVICE)
            t_label = t_label.to(DEVICE)
            domain_label = domain_label.long().to(DEVICE)

        _, d_pred = model(t_img, alpha)
        trg_d_loss = d_crit(d_pred, domain_label)

        loss = src_l_loss + src_d_loss + trg_d_loss

        loss.backward()
        optimizer.step()

        i += 1

        print ('epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' \
              % (epoch, i, len_dataloader, src_l_loss.cpu().data.numpy(),
                 src_d_loss.cpu().data.numpy(), trg_d_loss.cpu().data.numpy()))





/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch: 0, [iter: 1 / all 469], err_s_label: 2.350396, err_s_domain: 0.867046, err_t_domain: 0.681599
epoch: 0, [iter: 2 / all 469], err_s_label: 2.697859, err_s_domain: 0.970745, err_t_domain: 1.113457
epoch: 0, [iter: 3 / all 469], err_s_label: 2.527394, err_s_domain: 1.397564, err_t_domain: 1.315235
epoch: 0, [iter: 4 / all 469], err_s_label: 2.566563, err_s_domain: 1.052904, err_t_domain: 0.883873
epoch: 0, [iter: 5 / all 469], err_s_label: 2.392386, err_s_domain: 0.805111, err_t_domain: 0.752191
epoch: 0, [iter: 6 / all 469], err_s_label: 2.537903, err_s_domain: 0.758105, err_t_domain: 0.661590
epoch: 0, [iter: 7 / all 469], err_s_label: 2.370311, err_s_domain: 0.773212, err_t_domain: 0.704899
epoch: 0, [iter: 8 / all 469], err_s_label: 2.364611, err_s_domain: 0.748036, err_t_domain: 0.712834
epoch: 0, [iter: 9 / all 469], err_s_label: 2.248006, err_s_domain: 0.744881, err_t_domain: 0.694658
epoch: 0, [iter: 10 / all 469], err_s_label: 2.239871, err_s_domain: 0.733762, err_t_domain

KeyboardInterrupt: ignored

## DANN Test

In [42]:

train_transform = [transforms.ToTensor(), transforms.Normalize(0.5, 0.5), transforms.Resize((32,32))]
svhn_transform = [transforms.ToTensor(), transforms.Normalize(0.5, 0.5), transforms.Resize((32,32)), transforms.Grayscale(num_output_channels=1)]
train_mnist = datasets.MNIST(root='./data', train=False, transform=transforms.Compose(train_transform), download=True)
train_svhn = datasets.SVHN(root='./data', split='test', transform=transforms.Compose(svhn_transform), download=True)

mnist_loader = DataLoader(train_mnist, batch_size=32, shuffle=True)
svhn_loader = DataLoader(train_svhn, batch_size=32, shuffle=True)


len_dataloader = min(len(mnist_loader), len(svhn_loader))
# data_target_iter = iter(svhn_loader)
data_target_iter = iter(mnist_loader)

i = 0
n_total = 0
n_correct = 0
with torch.no_grad():    
    while i < len_dataloader:
        img, label = data_target_iter.next()
        bs = img.size(0)
        if torch.cuda.is_available():
            img = img.to(DEVICE)
            label = label.to(DEVICE)
        
        class_output, _ = model(img, alpha)

        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(label.data.view_as(pred)).cpu().sum()
        n_total += bs


        i += 1

print ('Target Acc: %f' \
    % (n_correct/n_total) )



Using downloaded and verified file: ./data/test_32x32.mat


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Target Acc: 0.717200
